# Moon Cycles — Trading Debug Notebook

This notebook exists for one practical reason:

- In the main Moon notebook you can see **good base trading metrics** (using `pred_label` as a signal)
  but the **parameter sweep prints zeros** (`ret=0`, `UI=0`, `win=nan`).

That looks like a bug, but in most cases it is simply this:

- The sweep uses a **neutral-zone signal** built from `pred_proba_up` with thresholds like `0.55/0.45`.
- For Moon/ephemeris models (especially XGBoost), probabilities often live in a very tight band around `0.50`.
- If almost all probabilities are between `0.45` and `0.55`, then the neutral-zone signal becomes **all NaN** ("no signal").
- With "no signal" every day, the strategy never enters a position, so equity stays at 1.0 => **return = 0**.

So the goal here is to make that behavior *explicit* and *easy to diagnose*.

We will:
1. Load the same `best_runs` as in the main notebook (using cache).
2. For each model, print a clear summary of:
   - probability distribution
   - signal coverage (UP / DOWN / NaN)
   - number of trades
3. Run trading backtests in 3 modes:
   - **Always decide** (signal = `pred_label`)
   - **Fixed neutral zone** (for demonstration: shows why it can produce 0 trades)
   - **Validation-quantile neutral zone** (a safer way to create "no-signal" while still having trades)

Honesty note:
- We do **not** optimize trading parameters on the test set.
- If we choose thresholds, we choose them using **validation** statistics, then we apply them to test.

In [ ]:
# ------------------------------
# Imports and project path setup
# ------------------------------
#
# We keep this notebook self-contained.
# It can be run after the main Moon notebook finishes.
#
# Important: adding PROJECT_ROOT to sys.path allows us to import our local modules.

from pathlib import Path
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

PROJECT_ROOT = Path('/home/rut/ostrofun')
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from RESEARCH2.Moon_cycles.moon_data import (
    MoonLabelConfig,
    build_moon_phase_features,
    load_market_slice,
)
from RESEARCH2.Moon_cycles.ephemeris_data import (
    EphemerisFeatureConfig,
    build_ephemeris_feature_set,
)
from RESEARCH2.Moon_cycles.bakeoff_utils import run_moon_model_bakeoff

from RESEARCH2.Moon_cycles.eval_visuals import VisualizationConfig
from RESEARCH2.Moon_cycles.trading_utils import (
    TradingConfig,
    backtest_long_flat_signals,
    build_signal_from_proba,
    plot_backtest_price_and_equity,
    sweep_trading_params,
)

pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 160)

In [ ]:
# ------------------------------
# Configuration (match the main notebook)
# ------------------------------
#
# If you want apples-to-apples results, keep these values the same as in
# `moon_cycles_deep_search_v2.ipynb`.

START_DATE = '2017-11-01'
END_DATE = None
USE_CACHE = True
VERBOSE = True

# Feature mode:
# - 'moon_only'     : strict Moon-only test
# - 'ephemeris_all' : maximum ephemeris information
FEATURE_SET = 'ephemeris_all'
CACHE_NAMESPACE = 'research2_moon' if FEATURE_SET == 'moon_only' else 'research2_ephem'

# Ephemeris config (used only when FEATURE_SET='ephemeris_all').
EPHEM_CFG = EphemerisFeatureConfig(
    coord_mode='both',
    orb_mult=0.25,
    include_pair_aspects=True,
    include_phases=True,
    add_trig_for_longitudes=True,
    add_trig_for_moon_phase=True,
    add_trig_for_elongations=True,
    exclude_bodies=(),
)

LABEL_CFG = MoonLabelConfig(
    horizon=1,
    move_share=0.5,
    label_mode='balanced_detrended',
    price_mode='raw',
)

# Gaussian grid (same as main notebook by default).
GAUSS_WINDOWS = [51, 101, 151, 201, 251, 301, 351, 401]
GAUSS_STDS = [10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 70.0, 90.0]

THRESHOLD_GAP_PENALTY = 0.25
THRESHOLD_PRIOR_PENALTY = 0.05

XGB_PARAMS = {
    'n_estimators': 500,
    'max_depth': 6,
    'learning_rate': 0.03,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'early_stopping_rounds': 50,
}

# Trading configuration.
FEE_RATE = 0.001  # 0.1% per buy and per sell
STOP_LOSSES = [0.0, 0.01, 0.02, 0.03, 0.05, 0.08]

# "Fixed" neutral zone (this is the one that often causes the "all zeros" sweep).
FIXED_UP_TH = 0.55
FIXED_DOWN_TH = 0.45

# Safer neutral zone idea (validation-quantile based):
# We will mark as signals only the extreme tails of validation probabilities.
# Example: q=0.20 means:
# - UP  if proba >= 80th percentile on validation
# - DOWN if proba <= 20th percentile on validation
# - else NaN (no signal)
VAL_QUANTILES = [0.10, 0.20, 0.30]

# Dark-theme visuals (same style as other research plots).
VIS_CFG = VisualizationConfig(
    rolling_window_days=90,
    rolling_min_periods=30,
    probability_bins=64,
)

print('Config loaded.')
print('FEATURE_SET      =', FEATURE_SET)
print('CACHE_NAMESPACE  =', CACHE_NAMESPACE)
print('Gaussian grid    =', len(GAUSS_WINDOWS) * len(GAUSS_STDS), 'combos')

In [ ]:
# -------------------------------------------
# Load market data + build astro features (cached)
# -------------------------------------------
#
# We need these only because `run_moon_model_bakeoff()` builds labels from market prices
# and aligns them with the feature matrix.

df_market = load_market_slice(
    start_date=START_DATE,
    end_date=END_DATE,
    use_cache=USE_CACHE,
    verbose=VERBOSE,
)

if FEATURE_SET == 'moon_only':
    df_features = build_moon_phase_features(
        df_market=df_market,
        use_cache=USE_CACHE,
        verbose=VERBOSE,
        progress=True,
    )
else:
    df_features = build_ephemeris_feature_set(
        df_market=df_market,
        cfg=EPHEM_CFG,
        cache_namespace=CACHE_NAMESPACE,
        use_cache=USE_CACHE,
        verbose=VERBOSE,
        progress=True,
    )

print('Market rows :', len(df_market))
print('Feature rows:', len(df_features))
print('Feature cols:', len([c for c in df_features.columns if c != 'date']))
print('Market range:', df_market['date'].min().date(), '->', df_market['date'].max().date())

In [ ]:
# -------------------------------------------
# Load best model runs (mostly from cache)
# -------------------------------------------
#
# This is the same call as in the main notebook.
# If cache is warm, it should be mostly file loads.

bakeoff = run_moon_model_bakeoff(
    df_market=df_market,
    df_moon_features=df_features,
    gauss_windows=GAUSS_WINDOWS,
    gauss_stds=GAUSS_STDS,
    label_cfg=LABEL_CFG,
    include_xgb=True,
    xgb_params=XGB_PARAMS,
    threshold_gap_penalty=THRESHOLD_GAP_PENALTY,
    threshold_prior_penalty=THRESHOLD_PRIOR_PENALTY,
    cache_namespace=CACHE_NAMESPACE,
    use_cache=USE_CACHE,
    verbose=VERBOSE,
)

best_by_val = bakeoff['best_by_val_table']
best_runs = bakeoff['best_runs']

print('Best-by-validation table (one row per model):')
display(best_by_val)

In [ ]:
# -------------------------------------------
# Helper: print probability + signal coverage diagnostics
# -------------------------------------------
#
# The *main* debugging question we answer:
# - Do we actually have any UP/DOWN signals after applying a neutral zone?
# - Or is the model so uncertain that everything becomes NaN?

def describe_proba_and_signal(df: pd.DataFrame, proba_col: str, signal_col: str, title: str) -> None:
    """Print easy-to-read diagnostics for one probability column and one signal column."""

    p = pd.to_numeric(df[proba_col], errors='coerce')
    sig = pd.to_numeric(df[signal_col], errors='coerce')

    # Probability distribution summary.
    desc = p.describe(percentiles=[0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99]).to_dict()

    # Signal coverage.
    n = len(sig)
    n_up = int((sig == 1).sum())
    n_down = int((sig == 0).sum())
    n_nan = int(sig.isna().sum())

    print() 
    print('-' * 100)
    print(title)
    print('Proba summary:', {k: float(v) for k, v in desc.items() if pd.notna(v)})
    print(
        f"Signal coverage: UP={n_up} ({n_up/max(n,1):.1%}), "
        f"DOWN={n_down} ({n_down/max(n,1):.1%}), "
        f"NO_SIGNAL(NaN)={n_nan} ({n_nan/max(n,1):.1%})"
    )

    if n_nan / max(n, 1) > 0.95:
        print('WARNING: >95% of days are NO_SIGNAL. This will usually create 0 trades and 0% return.')
        print('         Fix: use tighter thresholds (closer to 0.50) OR use validation-quantile thresholds.')


def build_neutral_signal_from_val_quantiles(
    pred_all_splits: pd.DataFrame,
    q_side: float,
    proba_col: str = 'pred_proba_up',
    out_col: str = 'trade_signal_neutral',
) -> tuple[pd.DataFrame, dict]:
    """Create a neutral-zone signal using thresholds computed from VALIDATION probabilities.

    Why this is useful:
    - For XGB Moon/ephemeris models, proba is often very close to 0.50.
    - A fixed 0.55/0.45 neutral zone can produce "no signal" for almost every day.
    - Quantile thresholds adapt to how "confident" the model actually is.

    What it does:
    - Compute thresholds from the validation distribution:
      down_th = quantile(q_side)
      up_th   = quantile(1 - q_side)
    - Apply those thresholds to ALL rows (train/val/test) so we can later slice.

    Returns:
    - a copy of `pred_all_splits` with `out_col` added
    - a small dict with the thresholds we used (for logging)
    """

    df = pred_all_splits.copy()
    p_val = pd.to_numeric(df[df['split_role'] == 'val'][proba_col], errors='coerce').dropna().to_numpy(dtype=float)

    if p_val.size < 10:
        raise ValueError('Not enough validation probabilities to compute quantiles.')

    q_side = float(q_side)
    down_th = float(np.quantile(p_val, q_side))
    up_th = float(np.quantile(p_val, 1.0 - q_side))

    # Safety: if thresholds collapse (can happen if proba is constant), fall back to 0.5/0.5.
    if not (down_th < up_th):
        down_th, up_th = 0.5, 0.5

    df[out_col] = build_signal_from_proba(
        df[proba_col].to_numpy(dtype=float),
        threshold_up=up_th,
        threshold_down=down_th,
    )

    return df, {'q_side': q_side, 'down_th': down_th, 'up_th': up_th}

In [ ]:
# -------------------------------------------
            # Trading debug run
            # -------------------------------------------
            #
            # We run this per model because different model families have different probability shapes.
            # For example:
            # - Some models can output very confident probabilities (close to 0 or 1).
            # - XGBoost here often outputs probabilities extremely close to 0.50.
            #   That does NOT mean it is useless, but it does mean "0.55/0.45 neutral zone" is too wide.

            for model_name, run in best_runs.items():
                print('
' + '=' * 120)
                print('MODEL:', model_name)

                pred = run['predictions'].copy()

                # We always slice TEST for performance reporting (honesty rule).
                test_df = pred[pred['split_role'] == 'test'].copy().reset_index(drop=True)
                test_df = test_df.dropna(subset=['close', 'pred_label', 'pred_proba_up'])

                # ------------------------------
                # A) Always-decide trading (signal = pred_label)
                # ------------------------------
                # This is the simplest possible translation from classification to trading:
                # - pred_label==1 => go long
                # - pred_label==0 => go flat
                test_df['trade_signal'] = test_df['pred_label'].astype(float)

                base_cfg = TradingConfig(
                    fee_rate=FEE_RATE,
                    stop_loss_pct=0.0,
                    exit_on_no_signal=False,
                    close_final_position=True,
                    initial_cash=1.0,
                )

                base_run = backtest_long_flat_signals(test_df, signal_col='trade_signal', cfg=base_cfg)
                print('Always-decide trading metrics:', base_run['metrics'])
                plot_backtest_price_and_equity(base_run, title=f'{model_name.upper()} - Trading (always decide)', vis_cfg=VIS_CFG)

                # Quick stop-loss sweep on the always-decide signal.
                # This sweep should NEVER print "all zeros", because we always have signals.
                sweep_base = sweep_trading_params(
                    df=test_df,
                    signal_col='trade_signal',
                    stop_losses=STOP_LOSSES,
                    exit_on_no_signal_options=(False,),  # irrelevant when there is no NaN
                    fee_rate=FEE_RATE,
                    close_final_position=True,
                    initial_cash=1.0,
                    verbose=True,
                )
                display(sweep_base['results_table'].head(10))

                # ------------------------------
                # B) Fixed neutral-zone (often too wide)
                # ------------------------------
                # This block shows EXACTLY why the sweep can produce 0 trades.
                test_df['trade_signal_fixed_neutral'] = build_signal_from_proba(
                    test_df['pred_proba_up'].to_numpy(dtype=float),
                    threshold_up=FIXED_UP_TH,
                    threshold_down=FIXED_DOWN_TH,
                )

                describe_proba_and_signal(
                    df=test_df,
                    proba_col='pred_proba_up',
                    signal_col='trade_signal_fixed_neutral',
                    title=f'Fixed neutral-zone diagnostics (UP_TH={FIXED_UP_TH}, DOWN_TH={FIXED_DOWN_TH})',
                )

                sweep_fixed = sweep_trading_params(
                    df=test_df,
                    signal_col='trade_signal_fixed_neutral',
                    stop_losses=STOP_LOSSES,
                    exit_on_no_signal_options=(False, True),
                    fee_rate=FEE_RATE,
                    close_final_position=True,
                    initial_cash=1.0,
                    verbose=True,
                )
                display(sweep_fixed['results_table'].head(10))

                # ------------------------------
                # C) Validation-quantile neutral-zone (more robust)
                # ------------------------------
                # Here we compute thresholds from validation distribution, then apply them to test.
                # This keeps the "no-signal" idea but avoids the "0 trades" trap.

                for q_side in VAL_QUANTILES:
                    pred_with_sig, th = build_neutral_signal_from_val_quantiles(
                        pred_all_splits=pred,
                        q_side=float(q_side),
                        proba_col='pred_proba_up',
                        out_col=f'trade_signal_valq_{q_side:.2f}',
                    )

                    test_df_q = pred_with_sig[pred_with_sig['split_role'] == 'test'].copy().reset_index(drop=True)
                    test_df_q = test_df_q.dropna(subset=['close', 'pred_proba_up'])

                    describe_proba_and_signal(
                        df=test_df_q,
                        proba_col='pred_proba_up',
                        signal_col=f'trade_signal_valq_{q_side:.2f}',
                        title=(
                            f"Val-quantile neutral-zone q_side={q_side:.2f} "
                            f"(DOWN_TH={th['down_th']:.4f}, UP_TH={th['up_th']:.4f})"
                        ),
                    )

                    sweep_q = sweep_trading_params(
                        df=test_df_q,
                        signal_col=f'trade_signal_valq_{q_side:.2f}',
                        stop_losses=STOP_LOSSES,
                        exit_on_no_signal_options=(False, True),
                        fee_rate=FEE_RATE,
                        close_final_position=True,
                        initial_cash=1.0,
                        verbose=True,
                    )

                    # Show only top-5 to keep output readable.
                    display(sweep_q['results_table'].head(5))

                    if sweep_q['best_run'] is not None:
                        plot_backtest_price_and_equity(
                            sweep_q['best_run'],
                            title=(
                                f"{model_name.upper()} - Trading (val-quantile neutral, q_side={q_side:.2f}) "
                                f"best by ulcer-adjusted return"
                            ),
                            vis_cfg=VIS_CFG,
                        )